In [6]:
from typing import List, Tuple, Dict

import numpy as np

In [8]:
# temporary object class
class DeepSortObject():
    id: int
    label: str
    confidence: float
    bbox: List[float]
    mask: np.ndarray

class ObjectTracker():
    _trajectory: Dict[int, np.ndarray] # obj.id -> np.ndarray of shape (3) # could be a np.ndarray of shape (t,3)
    def __init__(self):
        self.tracked_objects: Dict[int, Dict] = dict()  # obj.id -> trajectory
        self.tracked_kinematics: Dict[int, np.ndarray] = dict() # obj.id -> deltaX # QUESTION: Maybe we want to keep track of the last K deltas?

    def register_position(self, obj_key_:int, time_:int, position_:np.ndarray):
        self.tracked_objects[obj_key_][time_] = position_

        if time_ > 0:
            self.register_kinematics(obj_key_, time_, position_ - self.tracked_objects[obj_key_][time_ - 1])

    def register_kinematics(self, obj_key_:int, time_:int, dx_:np.ndarray):
        self.tracked_kinematics[obj_key_][time_] = dx_
    
    def predict_position(self, obj_key_:int, before_time_:int) -> np.ndarray:
        """
        Predicts the position of the object at before_time_ + 1
        args:
            obj_key_: int   -- object id
            time_: int      -- time step
        """
        assert obj_key_ in self.tracked_objects.keys(), f"Object {obj_key_} not found in tracker"
        assert before_time_ in self.tracked_objects[obj_key_].keys(), f"Position of object {obj_key_} not found at time {before_time_}"
        assert before_time_ in self.tracked_kinematics[obj_key_].keys(), f"Kinematics of object {obj_key_} not found at time {before_time_}"

        return self.tracked_objects.get(obj_key_).get(before_time_) + self.tracked_kinematics.get(obj_key_).get(before_time_)

def identify_objects(frame_:np.ndarray) -> Dict[int, DeepSortObject]:
    ...

def make_pointcloud(obj_mask_:np.ndarray, frame_:np.ndarray) -> np.ndarray:
    ...

In [ ]:
# global variables
FRAMES:List[(np.ndarray,np.ndarray)] = [] #TODO: implement get_frames()

DS_PREDICTION = True # True if prediction by DeepSort
KINEMATIC_PREDICTION = not DS_PREDICTION # True if prediction by kinematics

# dynamic variables
object_tracker = ObjectTracker()

lastFrameIds = {} # List of objects detected in last frame

for time_i, left_frame, right_frame in enumerate(FRAMES):
    # stereo feed and object identification
    objs_t = identify_objects(left_frame)

    for _past_obj in lastFrameIds:
        # object position estimation
        if _past_obj.id in objs_t.keys(): # object in last frame is in current frame
            obj_mask = objs_t.get(_past_obj.id).mask
            pc_obj = make_pointcloud(obj_mask, right_frame) # TODOWork from Alex
        else: # object in last frame is not in current frame (OCCLUSION)
            if KINEMATIC_PREDICTION:
                # kinematics estimation
                _pos_obj = object_tracker.predict_position(_past_obj.id, time_i)
            else:
                # deepsort estimation
                # get predicted bbox form deepsort
                raise NotImplementedError("DeepSort prediction not implemented")
                ...
            # object tracking
            object_tracker.register_position(_past_obj.id, time_i, _pos_obj)

    lastFrameIds = set(objs_t.keys())   



In [4]:
s1 = {1,2,3}
s2 = {2,3,4}

s3 = s1.difference(s2).union(s1.intersection(s2))

print(s3)

{1, 2, 3}
